In [ ]:
import sys
import os
import time

import ast
import csv
import math
import random
import copy
import numpy as np

import folium
from folium.plugins import MarkerCluster
import gurobipy as gp
from gurobipy import GRB
import networkx as nx
import matplotlib.pyplot as plt

### Lendo arquivos

In [ ]:
def read_input(filename):
    with open(filename, 'r') as f:
        data = f.readlines()
    
    n, m, p = map(int, data[0].split())
    edges = []
    
    for i in range(m):
        u, v, cost = map(int, data[i + 1].split())
        edges.append((u - 1, v - 1, cost))  # Convertendo para 0-index
    
    return n, edges, p

def floyd_warshall(n, edges):
    # Inicializando a matriz de custos com infinito
    inf = sys.maxsize
    cost = [[inf] * n for _ in range(n)]
    
    # Preenchendo a diagonal com 0
    for i in range(n):
        cost[i][i] = 0
    
    # Preenchendo a matriz de custos com os valores do arquivo
    for u, v, c in edges:
        cost[u][v] = c
        cost[v][u] = c
    
    # Algoritmo de Floyd-Warshall
    for k in range(n):
        for i in range(n):
            for j in range(n):
                if cost[i][j] > cost[i][k] + cost[k][j]:
                    cost[i][j] = cost[i][k] + cost[k][j]
    
    return cost


# Função para ler todas as distâncias do arquivo txt
def read_input_distances(filepath):
    with open(filepath, 'r') as f:
        lines = f.readlines()

    distancias = []
    d = []
    for line in lines:
        line = line.strip()
        if line.startswith("Instance"):
            if d:
                distancias.append(d)
                d = []
        elif line:
            row = list(map(float, line.split()))
            d.append(row)
    if d:  # adiciona a última
        distancias.append(d)

    return distancias

### Salvando todas as distâncias

In [ ]:
# Salvando todas as distâncias
base_directory = 'Base de Dados/OR-Library'
distancias = []

for i in range(40):
    start_time = time.time()
    filename = f'{base_directory}/pmed{i+1}.txt'
    if os.path.exists(filename):
        n, edges, P = read_input(filename)
    else:
        print(f'File {filename} does not exist.')
        continue
    
    result = floyd_warshall(n, edges)  # Algoritmo de Floyd-Warshall
    
    d = []
    for row in result:
        d.append(row)
    
    distancias.append(d)  # Armazena a matriz de distâncias d
    print(f"Instancia {i+1}, {time.time() - start_time}")

# Salvando as distâncias d em um arquivo de texto
arquivo = 'distancias_or_library.txt'
with open(arquivo, 'w') as f:
    for i, d in enumerate(distancias):
        f.write(f"Instance {i+1}:\n")
        for row in d:
            f.write(' '.join(map(str, row)) + '\n')
        f.write('\n')

print(f"Todas as distâncias foram salvas em '{arquivo}'.")

### Modelagem do Problema

In [ ]:
base_directory = 'Base de Dados/OR-Library'
distancias = read_input_distances("distancias_or_library.txt")
solutions = []

for i in range(0,40):
    filename = f'{base_directory}/pmed{i+1}.txt'
    if os.path.exists(filename):
        n, edges, P = read_input(filename)
    else:
        print(f'File {filename} does not exist.')
    
    #result = floyd_warshall(n, edges) # algoritmo de Floyd-Warshall

    d = distancias[i] 
    
    M = n # M = len(d)
    N = n # N = len(d[0])
    
    m = gp.Model("Problema de Localização de Facilidades")
    
    # Variáveis de decisão contínuas: x[i, j] representa o quanto o cliente i está atendido pelo posto j
    x = m.addVars(M, N, vtype=GRB.CONTINUOUS, lb=0, name="Facilidade")
    
    # Variáveis binárias: y[j] = 1 se o posto j for escolhido, 0 caso contrário
    y = m.addVars(N, vtype=GRB.BINARY, name="Aberto")
    
    # Função objetivo: minimizar a distância total ponderada
    m.setObjective(gp.quicksum(d[i][j] * x[i, j] for i in range(M) for j in range(N)), GRB.MINIMIZE)
    
    # Restrição 1: cada cliente deve ser atendido exatamente por um posto
    m.addConstrs((gp.quicksum(x[i, j] for j in range(N)) == 1 for i in range(M)), name="DemandaClientes")
    
    # Restrição 2: número de instalações limitadas
    m.addConstr(gp.quicksum(y[j] for j in range(N)) == P, name="NumeroInstalacoes")
    
    # Restrição 3: cliente só pode ser atendido por um posto se ele estiver aberto
    m.addConstrs((x[i, j] <= y[j] for i in range(M) for j in range(N)), name="ClientePorPosto")
    
    m.setParam('OutputFlag', 0) # Desativar saída do Gurobi
    m.setParam('TimeLimit', 1800) # Limite de tempo de execução do modelo: 30 minutos
    start_time = time.time()
    m.optimize()
    execution_time = time.time() - start_time
    
    # Armazenando a solução, limites, gap e tempo de execução
    primal_bound = m.objVal
    dual_bound = m.ObjBound
    gap = m.MIPGap
    solutions.append({
        'Primal Bound': primal_bound,
        'Dual Bound': dual_bound,
        'Gap': gap,
        'Execution Time': execution_time
    })

    print(f"Tempo de Execução: {execution_time}")
    print(f"Instance {i+1}: Custo mínimo total: {str(m.objVal)}")
    
    fornecedores_abertos = [j for j in range(N) if y[j].X > 0.5]
    print("Fornecedores abertos:", fornecedores_abertos)
    print('\n')
    
# Imprime todas as soluções armazenadas
print("\nTodas as soluções:")
for i, solution in enumerate(solutions):
     print(f"Instance {i+1}: Primal Bound = {solution['Primal Bound']}, Dual Bound = {solution['Dual Bound']}, Gap = {solution['Gap']}, Execution Time = {solution['Execution Time']}")

In [ ]:
# Salva os dados de solutions em um arquivo txt
output_file = 'dados_solutions.txt'
with open(output_file, 'w') as file:
    for solution in solutions:
        file.write(f"{solution}\n")

print(f"Soluções foram salvas em {output_file}.")

In [ ]:
lista_solution = []

for i, solution in enumerate(solutions):
    lista_solution.append(solution['Dual Bound'])

## VND

In [ ]:
# Estratégia gulosa para gerar solução inicial
def metodo_guloso(n, P, d):
    # Inicialmente, nenhum fornecedor está aberto
    opened = []
    remaining = list(range(n))
    
    # Inicializar vetor de alocação com distâncias infinitas
    customer_assign_costs = [float('inf')] * n
    
    for _ in range(P):
        best_facility = None
        best_improvement = float('inf')
        
        for j in remaining:
            total_cost = 0
            for i in range(n):
                # Para cada cliente, considera a melhor distância se j for adicionado
                total_cost += min(customer_assign_costs[i], d[i][j])
            if total_cost < best_improvement:
                best_improvement = total_cost
                best_facility = j
        
        opened.append(best_facility)
        remaining.remove(best_facility)
        # Atualiza os custos de alocação com o novo fornecedor
        for i in range(n):
            customer_assign_costs[i] = min(customer_assign_costs[i], d[i][best_facility])

    s0 = [1 if j in opened else 0 for j in range(n)]
    return s0
    

def modelo(d, M, N):
    m = gp.Model()
    m.setParam('OutputFlag', 0)
    m.setParam("MIPGap", 0.05)
    m.setParam("TimeLimit", 60)

    x = m.addVars(M, N, vtype=GRB.CONTINUOUS, lb=0, name='x')
    ativos = m.addVars(N, vtype=GRB.BINARY, name='ativos')  # será atualizado dinamicamente

    # Função objetivo
    m.setObjective(gp.quicksum(d[i][j] * x[i, j] for i in range(M) for j in range(N)), GRB.MINIMIZE)

    # Cada cliente deve ser atendido por uma instalação
    for i in range(M):
        m.addConstr(gp.quicksum(x[i, j] for j in range(N)) == 1)

    # Ligação x[i,j] <= ativos[j]
    restricoes_uso = {}
    for i in range(M):
        for j in range(N):
            restricoes_uso[i, j] = m.addConstr(x[i, j] <= ativos[j], name=f"restricao_uso_{i}_{j}")

    return m, x, ativos

def f(m, ativos, s):
    for j in range(len(s)):
        ativos[j].LB = s[j]
        ativos[j].UB = s[j]
    m.update()
    m.optimize()
    return m.objVal

# Vizinhança Swap: troca de dois postos abertos (swap entre 1 e 0)
def vizinhanca_v1(y_):
    vizinhos = []
    abertos = [i for i, val in enumerate(y_) if val == 1]
    fechados = [i for i, val in enumerate(y_) if val == 0]
    for i in abertos:
        for j in fechados:
            novo = y_.copy()
            novo[i], novo[j] = 0, 1
            vizinhos.append(novo)
    return vizinhos

# Vizinhança olha posições antecessora e sucessora da solução para possível troca
def vizinhanca_v3(y_):
    vizinhos = []
    n = len(y_)
    
    for i in range(n):
        # Tenta trocar com a posição anterior
        if i > 0 and y_[i] != y_[i - 1]:
            novo = y_.copy()
            novo[i], novo[i - 1] = novo[i - 1], novo[i]
            vizinhos.append(novo)
        # Se não puder trocar com anterior, tenta sucessor
        elif i < n - 1 and y_[i] != y_[i + 1]:
            novo = y_.copy()
            novo[i], novo[i + 1] = novo[i + 1], novo[i]
            vizinhos.append(novo)
        # Caso contrário, não adiciona vizinho

    return vizinhos

def vizinhanca_v7(y_):
    vizinhos = []
    abertos = [i for i, val in enumerate(y_) if val == 1]
    fechados = [i for i, val in enumerate(y_) if val == 0]

    for i in abertos:
        if not fechados:
            break  # não há para onde trocar
        j = random.choice(fechados)
        novo = y_.copy()
        novo[i], novo[j] = 0, 1
        vizinhos.append(novo)
        fechados.remove(j)  # evita usar o mesmo 0 novamente (opcional)

    return vizinhos

# vizinhanca troca o primeiro posto aberto pelos len(y_)/sum(y_) ultimos postos fechados
def vizinhanca_v9(y_):
    vizinhos = []
    primeiro_aberto = next((i for i, val in enumerate(y_) if val == 1), None)
    
    if primeiro_aberto is None:
        return []  # Não há postos abertos para trocar

    for j in reversed(range(len(y_))):
        if y_[j] == 0:
            novo = y_.copy()
            novo[primeiro_aberto] = 0
            novo[j] = 1
            vizinhos.append(novo)

    return vizinhos
    
def VND(s0, vizinhancas, m, ativos, d, M, N, P):
    tempo_max = 3600
    tempo_limite_vizinhanca = 900
    s = s0
    f_s = f(m, ativos, s)
    k = 0

    inicio = time.time()
    while k < len(vizinhancas) and (time.time() - inicio) < tempo_max:
        start_time = time.time()
        vizinhos = vizinhancas[k](s)

        melhor = s
        f_melhor = f_s
        encontrou_melhor = False

        print(f"Tamanho da vizinhança {k}: {len(vizinhos)}")

        for v in vizinhos:
            if time.time() - start_time > tempo_limite_vizinhanca:
                print(f"Tempo excedido na vizinhança {k}")
                break

            f_v = f(m, ativos, v)

            if f_v < f_melhor:
                melhor = v
                f_melhor = f_v
                encontrou_melhor = True
                break

        if encontrou_melhor:
            s = melhor
            f_s = f_melhor
            k = 0
        else:
            k += 1

    return s, f_s

In [ ]:
base_directory = 'Base de Dados/OR-Library'
distancias = read_input_distances("distancias_or_library.txt")
solucoes = []

for i in range(0,40):
    filename = f'{base_directory}/pmed{i+1}.txt'
    if os.path.exists(filename):
        n, arestas, P = read_input(filename)
    else:
        print(f'File {filename} does not exist.')

    d = distancias[i] 
    
    M = n
    N = n

    # ESTRATÉGIA ALEATÓRIA
    # posicoes = random.sample(range(N), P) 
    # s0 = [1 if j in posicoes else 0 for j in range(N)]

    # ESTRATÉGIA GULOSA
    start_time = time.time()
    s0 = metodo_guloso(n, P, d)
    print(f"Tempo de execução Método Guloso: {time.time() - start_time:.2f} segundos")
    
    fornecedor_inicial = [j for j in range(N) if s0[j] > 0.5]
    print("Fornecedores inicial:", fornecedor_inicial)

    # Criar modelo e variáveis globais de Gurobi
    m, x, ativos = modelo(d, M, N)
    
    vizinhancas = [vizinhanca_v9, vizinhanca_v7, vizinhanca_v3, vizinhanca_v1]
    
    start_time = time.time()
    melhor_solucao, custo = VND(s0, vizinhancas, m, ativos, d, M, N, P)
    execution_time = time.time() - start_time

    solucoes.append({
        'Custo': custo,
        'Execution Time': execution_time
    })

    print(f"Tempo de Execução do VND: {execution_time:.2f} segundos")
    print(f"Instance {i+1}: Custo mínimo total: {custo:.1f}")
    #print(f"Instance {i+1}: Custo mínimo total: {custo:.1f}")
    fornecedores_abertos = [j for j in range(N) if melhor_solucao[j] > 0.5]
    print("Fornecedores abertos:", fornecedores_abertos)
    print("\n")

In [ ]:
# Salva os dados de solucao em um arquivo txt
output_file = 'dados_solucoes.txt'
with open(output_file, 'w') as file:
    for solucao in solucoes:
        file.write(f"{solucao}\n")

print(f"Soluções foram salvas em {output_file}.")

### Abrindo arquivos com os dados

In [ ]:
solutions = []
solucoes = []

with open("dados_solutions.txt", "r") as file:
    for line in file:
        if line.strip():  # ignora linhas em branco
            solutions.append(ast.literal_eval(line.strip()))

with open("dados_solucoes_aleatorias.txt", "r") as file:
    for line in file:
        if line.strip():  # ignora linhas em branco
            solucoes.append(ast.literal_eval(line.strip()))

In [ ]:
for i, solution in enumerate(solutions):
    for j, solucao in enumerate(solucoes):
        if i == j:
            print(f"gap e tempo pmed{j+1}: {(100 * (solucao['Custo'] - solution['Dual Bound']) / solucao['Custo']):.2f}, {solucao['Execution Time']:.2f}")

In [ ]:
# instância 1
pmed1 = [5819.0, 5819.0, 5819.0, 5819.0, 5819.0, 5819.0, 5819.0, 5819.0, 5819.0, 5819.0]
tempo1 = [85.81, 81.25, 78,53, 97.22, 91.85, 95.18, 133.34, 74.70, 87.11, 82.97]

media_p = sum(pmed1) / len(pmed1)
desvio_medio_p = sum(abs(x - media_p) for x in pmed1) / len(pmed1)

media_t = sum(tempo1) / len(tempo1)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tmepo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 2
pmed = [4105, 4093.0, 4093.0, 4093.0, 4093.0, 4105.0, 4105.0, 4093.0, 4093.0, 4093.0]
tempo= [186.00, 192.17, 192.19, 198.73, 197.21, 277.09, 190.92, 196.23, 216.62, 160.51]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tmepo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 3
pmed = [4250, 4250, 4250, 4250, 4250, 4250, 4270, 4250, 4250, 4250]
tempo = [323.54, 277.68, 276.19, 289.99, 274.60, 295.65, 291.14, 181.33, 266.23, 195.22]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 4
pmed4 = [3038, 3038, 3034, 3034, 3034, 3034, 3034, 3038, 3057, 3082]
tempo4 = [516.32, 553.56, 433.31, 464.51, 537.65, 240.12, 357.13, 315.36, 261.97, 286.09]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 5
pmed = [1358, 1373, 1355, 1373, 1373, 1355, 1355, 1373, 1355, 1355]
tempo = [678.73, 989.95, 492.31,  603.92, 536.42, 632.89, 745.33, 701.83, 697.34, 499.59]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 6
pmed = [7824, 7824, 7824, 7824, 7824, 7824, 7824, 7824, 7824, 7824]
tempo = [623.10,  527.81, 909.27, 613.91, 665.78, 847.81, 639.72, 738.25, 679.11, 622.10]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 7
pmed = [6232.0, 5631, 5631, 5631, 5631, 5635, 5678, 5921, 5634, 6232,]
tempo = [1421.93, 1035.71, 1009.31, 1677.90, 1287.14, 1661.53, 1211.97, 1037.28, 1457.56, 1195.14]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 8
pmed = [4445, 4447, 4563, 4478, 4446, 4447, 4456, 4451, 4448, 4449]
tempo = [2548.09, 2331.90, 2876.44, 2965.40, 2611.32, 2774.63, 2573.19, 2674.49, 2854.33, 2634.80]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 9
pmed = [2788,  2790, 2803, 2807, 2790, 2789, 2799, 2792, 2809, 2788]
tempo = [2756.60,  2845.36, 2678.99, 2510.13, 2876.44, 2963.27, 2913.22, 2764.44, 2658.56, 2301.75]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 10
pmed = [1289.0, 1266, 1290, 1291, 1283, 1272, 1278, 1276, 1293, 1289]
tempo = [3998.66,  3988.04, 3845.62, 3967.13, 3799.18, 3856.64, 3846.63, 3867.44, 3968.78, 3997.45]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 11
pmed = [7696, 7723, 8145, 8241, 7698, 7699, 7696, 8023, 7696, 8242.0]
tempo = [1392.65,  745.18, 997.23, 1458.63, 1247.63, 902.75, 1087.23, 1225.21, 1365.45, 1000.98]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 12
pmed = [6660.0,  6634, 6641, 6634, 6660, 6654, 6653, 6653, 6634, 6649]
tempo = [2146.74,  3816.07, 3625.12, 3689.45, 2254.66, 2013.17, 3478.77, 2987.54, 2247.86, 2354.12]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 13
pmed = [4605, 4656.0, 4605, 4605, 4605, 4605, 4605, 4605, 4605, 4605, 4605, 4656 ]
tempo = [3813.54, 3914.95, 3854.16, 3698.77, 3689.45, 3845.12, 3885.14, 3800.96, 3746.03, 3600.18]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 14
pmed = [3105.0, 3102, 3101, 3102, 3225, 3114, 3185, 3105, 3165, 3663]
tempo = [3610.92, 3935.72, 3875.16, 3631.45, 3763.12, 3902.23, 3764.56, 3647.13, 3698.78, 3777.14]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 15
pmed = [1815, 1815, 1815, 1819, 1823, 1920, 1878, 1844, 1845, 1834]
tempo = [3837.21, 3699.73, 3789.25, 3471.45, 3953.14, 3602.33, 3612.78, 3945.12, 3884.65, 3706.14]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 16
pmed = [8162,  8582.0, 8326, 8178, 8198, 8165, 8378, 8301, 8264, 8200]
tempo = [2271.38,  2393.52, 3458.45, 2391.13, 2443.01, 2985.63, 2651.22, 2145.62, 2356.45, 2184.75]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 17
pmed = [8051.0, 7090, 7125, 7671, 7095, 7104, 7090, 7091, 7850, 7543, 7544]
tempo = [4077.35, 3997.74, 3856.55, 3968.03, 3908.43, 3746.68, 3901.23, 4123.56, 3875.42, 3865.41]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 18
pmed = [5169,  5201, 5198, 5170, 5169, 5169, 5198, 5173, 5183, 5183]
tempo = [3609.37, 3645.55, 3633.30, 3786.41, 3649.12, 3942.66, 3801.46, 3602.41, 3622.78, 3846.60 ]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 19
pmed = [3355,  3459.0, 3363, 3423, 3417, 3498, 3394, 3376, 3372, 3402]
tempo = [3645.82,  3625.02, 3698.15, 3748.84, 3683.23, 3887.73, 3648.13, 3613.22, 3699.01, 3745.41]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 20
pmed = [2193, 2298, 2298.0, 2298.0, 2193, 2193, 2193, 2193, 2193, 2193]
tempo = [3634.00, 3621.66, 3658.74, 3681.22, 3709.84, 3697.30, 3631.54, 3698.45, 3678.20, 3751.86]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 21
pmed = [9179.0, 9138, 9179.0, 9179.0, 9179.0, 9179.0, 9179.0, 9179.0, 9138, 9138]
tempo = [3684.71, 3629.66, 3775.75, 3681.22, 3709.84, 3797.30, 3631.54, 3754.96, 3684.19, 3705.31]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 22
pmed22 = [8730.0, 9005, 9135, 9013, 8854, 8960, 9003, 8730, 8933, 9511]
tempo22 = [3616.93, 3749.39, 3697.07, 3974, 56, 4069.22, 3625.46, 3673.93, 3874.68, 3841.17, 3931.10]

media_p = sum(pmed22) / len(pmed22)
desvio_medio_p = sum(abs(x - media_p) for x in pmed22) / len(pmed22)

media_t = sum(tempo22) / len(tempo22)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 23
pmed23 = [5368.0, 5433, 5568, 5478, 5463, 5441, 6096, 5567, 5402, 5399]
tempo23 = [3635.66, 3697.20, 3645.63, 3687.99, 3637.52, 3794.50, 3761.33, 3798.33, 3608.95, 3867.13]

media_p = sum(pmed23) / len(pmed23)
desvio_medio_p = sum(abs(x - media_p) for x in pmed23) / len(pmed23)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 24
pmed = [3793.0,  3826, 3799, 3823, 3841, 3846, 3798, 3853, 3865, 3850]
tempo = [3638.8,  3640.31, 3784.64, 3821.10, 3697.31, 3764.65, 3745.25, 3648.13, 3613.99, 3842.12]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 25
pmed = [2738, 2594.0, 2643, 2594.0, 2594.0, 2594.0, 2890, 2654, 2673, 2797]
tempo = [3677.87, 3677.87, 3654.80, 3714.6, 3678.33, 3601.52, 3763.12, 3814.73, 3691.62, 3811.90]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 26
pmed = [10444, 10336.0, 10543, 10339, 10497, 11001, 10523, 10519, 11170, 10500]
tempo = [3727.13, 3652.23, 3614.36, 3796.34, 3772.12, 3676.42, 3637.09, 3977.63, 3612.26, 3677.61]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 27
pmed = [8864, 9180.0, 8865.0, 8870, 8893, 8887, 8872, 8991, 8865, 9003]
tempo = [3714.52,  3714.43, 3816.63, 3712.93, 3967.17, 3914.19, 3717.13, 3863.72, 4012.25, 3675.77]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 28
pmed = [5813.0, 5457, 5556.0, 5813.0, 5813.0, 5813.0, 5457, 5457, 5457, 5457]
tempo = [3628.16, 3788.63, 3656.83, 3627.16, 3756.26, 3703.25, 3628.16, 3623.00, 3770.66, 3641.56]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 29
pmed = [3859, 3922.0, 3922.0, 3922.0, 3859, 3922.0, 3922.0, 3859, 3922.0, 3922.0]
tempo = [3772.27, 3663.80, 3654.25, 3689.52, 3723.45, 3756.49, 3689.54, 3702.54, 3623.65, 3661.01]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 30
pmed = [2979.0, 2930.0, 2930.0, 2930.0, 2930, 2979.0, 2951.0, 2930.0, 2930.0, 2930.0]
tempo = [3753.22, 3770.79, 3705.57, 3699.23, 3753.0, 3970.11, 3662.89, 3708.58, 3698.75, 3729.03]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 31
pmed = [10733, 10612.0, 10612.0, 10612.0, 10612.0, 10612.0, 10612.0, 10612.0, 10733, 10733]
tempo = [3721.03, 3777.85, 3723.68, 3798.56, 3699.84, 3745.65, 3776.88, 3708.91, 3739.45, 3675.27]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 32
pmed = [10468.0, 10263, 10263, 10263, 10263, 10263, 10263, 10468.0, 10468.0, 10468.0]
tempo = [3966.27, 3928.45, 3857.76, 3756.23, 3699.64,  3844.26, 3647.68, 3800.67, 3646.99, 3788.66]


media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 33
pmed = [5591.0, 5660, 5591.0, 5591.0, 5660, 5660, 5660, 5591.0, 5591.0, 5591.0]
tempo = [3666.69, 3899.47, 3745.66, 3631.89, 3627.91, 3610.11, 3898.61, 3764.39, 3689.23, 3814.53]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 34
pmed = [4091.0, 4335, 4091.0, 4091.0, 4091.0, 4335, 4256, 4159, 4335, 4091]
tempo = [3765.18, 3791.57, 3810.54, 3871.45, 3627.95, 3718.51, 3798.61, 3664.72, 3759.03, 3674.57]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 35
pmed = [11433.0, 11908, 12694.0, 11433.0, 11433.0, 11433.0, 11564.0, 11433.0, 11908, 11908]
tempo = [3712.01, 4108.59, 4154.13, 3875.26, 3697.54, 3745.31, 3628.97, 3911.08, 3734.68, 3931.27]


media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 36
pmed = [11798.0, 11535, 11535, 11535, 11535, 11535, 11535, 11535, 11535, 11535, 11798.0, 11798.0]
tempo = [4252.37, 3618.90, 3789.65, 3875.63, 3912.37, 3700.52, 3731.82, 3962.34, 4087.96, 3999.14]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}"))

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 37
pmed = [6761, 6695.0, 6695.0, 6695.0, 6695.0, 6695.0, 6695.0, 6695.0, 6695.0, 6761, 6761]
tempo = [3890.89, 3991.16, 3865.23, 3749.22, 3943.00, 3783.29, 3933.07, 3746.15, 3930.85, 3855.62]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 38
pmed = [12341.0, 12173, 12173, 12173, 12173, 12341, 12327.0, 12327.0, 12327.0, 12327.0]
tempo = [3733.91, 3686.42, 3784.56, 3692.65, 3755.10, 3867.96, 3710.52, 3666.2, 3821.58, 3986.57]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 39
pmed = [11057.0, 10580, 10580, 11057.0, 10580, 10580, 10580, 10580.0, 11057.0, 10580]
tempo = [3815.21, 3915.51, 3865.30, 3776.45, 3706.67, 3699.98, 3745.56, 3789.65, 3721.10, 3756.66, 3986.47]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)

In [ ]:
# instância 40
pmed = [6694, 6618.0, 6793, 6699, 6703, 6781, 6753, 6701, 6695, 6719]
tempo = [3660.44, 3753.94, 3845.13, 3661.80, 3746.54, 3691.25, 3967.44, 3641.11, 3675.45, 3699.31]

media_p = sum(pmed) / len(pmed)
desvio_medio_p = sum(abs(x - media_p) for x in pmed) / len(pmed)

media_t = sum(tempo) / len(tempo)

print(f"Desvio médio p: {desvio_medio_p}")
print(f"Tempo médio t: {media_t}")

porcentagem = 100 * (desvio_medio_p/media_p)
print('Porcentagem do desvio médio: ', porcentagem)